# [CMI-SleepState-Detection](https://www.kaggle.com/competitions/child-mind-institute-detect-sleep-states/overview)
## Child Mind Institute - Detect Sleep States
### Detect sleep onset and wake from wrist-worn accelerometer data
_______________________________________________________________________ 
# Author Details:
- Name: Najeeb Haider Zaidi
- Email: zaidi.nh@gmail.com
- Profiles: [Github](https://github.com/snajeebz)  [LinkedIn](https://www.linkedin.com/in/najeebz) [Kaggle](https://www.kaggle.com/najeebz)
- License: Private, Unlicensed, All the files in this repository under any branch are Prohibited to be used commercially or for personally, communally or privately unless permitted by author in writing.
- Copyrights 2023-2024 (c) are reserved only by the author: Najeeb Haider Zaidi
________________________________________________________________________
# Attributions:
The Dataset has been provided by Child Mind Institute. in [Kaggle Competition](https://www.kaggle.com/competitions/child-mind-institute-detect-sleep-states/overview) which the author is participating in and authorized to use the dataset solely for the competition purposes.
________________________________________________________________________

In [ ]:
!pip install pandarallel


In [ ]:
import numpy as np # linear algebra
import pandas as pd# data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
from pandarallel import pandarallel
import plotly.express as px
import matplotlib.pyplot as plt
from datetime import datetime as dts
pd.set_option('display.max_row', 500)
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
import warnings
warnings.filterwarnings("ignore")
pd.set_option('display.max_colwidth', None)
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        
from tqdm import tqdm
tqdm.pandas()
pandarallel.initialize(progress_bar=True)

In [ ]:
events=pd.read_csv('/kaggle/input/child-mind-institute-detect-sleep-states/train_events.csv')

In [ ]:
df_full=pd.read_pickle('/kaggle/input/later-data/full_clustered .pkl')

# Analyzing the relationshiip between sleep and cluster

In [ ]:
df_sleep=df.loc[df['sleep']==1]
df_onset=df.loc[df['sleep']==0]

In [ ]:
df_sleep['cluster'].mode()

In [ ]:
df_onset['cluster'].mode()

In [ ]:
#df['timestamp']=df['timestamp'].parallel_apply(lambda x: pd.to_datetime(x,utc=True))

In [ ]:
df.info()

In [ ]:
df.to_pickel('cluster_timestamped.pkl')

In [ ]:
events

In [ ]:
events['step'].isnull().sum()

In [ ]:
event_df=events.dropna()

In [ ]:
event_df

In [ ]:
df['window']=np.nan

In [ ]:
ind=df.index[df['event'].isna()==False]
print(ind)

In [ ]:
# 30 min window on change event, with single timestep:5sec. 
c=0
for i in tqdm(ind):
    a=i-360
    b=i+360
    df['window'].loc[a:i]=int(c)
    c=c+1
    df['window'].loc[i:b]=int(c)
    c=c+1
print('Windows Created: ', c)    



In [ ]:
df=df[df['window'].isna()==False]

In [ ]:
df['timestamp']=df['timestamp'].parallel_apply(lambda x: pd.to_datetime(x,utc=True))

# Adding rolling Deviation

In [ ]:
def prepare(df):
    df['sd_anglez_1']=pd.to_numeric(df_n['anglez'].rolling(window=12).std())
    df['sd_enmo_1']=pd.to_numeric(df_n['enmo'].rolling(window=12).std())
    df['sd_anglez_1'].fillna('0.0',inplace=True)
    df['sd_enmo_1'].fillna('0.0',inplace=True)
    return(df)
    

## Analyzing the windows

In [ ]:
def visualization(df_n):
    for i in range(10):
        title="Sleep: "+ str(df_n['sleep'][df_n['window']==i].mean())
        fig=px.line(y=df_n['sleep'][df_n['window']==i],x=df_n['timestamp'][df_n['window']==i])
        fig.add_scatter(y=df_n['cluster'][df_n['window']==i],x=df_n['timestamp'][df_n['window']==i], name='cluster')
        fig.add_scatter(y=df_n['enmo'][df_n['window']==i],x=df_n['timestamp'][df_n['window']==i], name='sleep')
        fig.update_layout(title=title)
        fig.show()

In [ ]:
df['window'].isnull().sum()

# Evaluate Prediction

In [ ]:
def evaluate(y_test,ypred)
    from sklearn.metrics import precision_score
    from sklearn.metrics import recall_score
    from sklearn.metrics import f1_score
    from sklearn.metrics import accuracy_score
    from sklearn.metrics import confusion_matrix
    print("Accuracy: ",accuracy_score(y_test,y_pred)) 
    print("Precision Score : ", precision_score(y_test,y_pred)) #precision measures the proportion of true positive predictions among all positive instances. how many of survived predicted actually survived, doesn't verifies 0's 70 survived as preicted whereas actually 92 survived so 70/92 will be the precision.  if we predicted 70 survived, so presion will tell how many of those 70 predicted survived matches the actual row by row data. It checkes all positives and verifies if the answer is true for each row?
    print("Recall Score: ", recall_score(y_test,y_pred, average='macro')) #Recall measures the proportion of true positive predictions among all actual positive instalnces. If we predicted 100 survived correctly whereas actually 100 survived out of which 67 predicted correctly so recall will be 0.67
    print("F1 Score: ",f1_score(y_test,y_pred)) #mean of recall and precision
    cm=confusion_matrix(y_test, y_pred)
    figure= px.imshow(cm,text_auto=True, width=1200, height=1200)
    figure.show()

In [ ]:
#df.to_pickle('ts_win_60_diff.pkl')
df=prepare(pd.read_pickle('ts_win_60_diff.pkl'))


In [ ]:
df.groupby(df['sleep']).describe().transpose()

# Normalizing cluster and sleep

In [ ]:
df_n=df


In [ ]:
from sklearn.model_selection import train_test_split
X=df_n[['sd_anglez_1','sd_enmo_1','anglez','enmo','cluster']]
y=df[['sleep_bool']]
X_train, X_test, y_train, y_test =train_test_split(X,y,test_size=0.2)

## Import Model

In [ ]:
from joblib import dump, load
model=load('rf_model.joblib')
y_pred=model.predict(X_test)
evaluate(y_test,y_pred)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rf =RandomForestClassifier(n_jobs=-1,verbose=1) 
print ('Training the model')
rf.fit(X_train,y_train)
print ('Saving the model')
from joblib import dump, load
dump(rf, 'rf_model.joblib')
y_pred=model.predict(X_test)
evaluate(y_test,y_pred)


In [ ]:
pred=y_test

In [ ]:
pred['predicted']=y_pred